<a href="https://colab.research.google.com/github/aleks-haksly/Simulative/blob/main/Python/%D0%9F%D0%B5%D1%80%D0%B2%D1%8B%D0%B9_%D0%BA%D0%B5%D0%B9%D1%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 кейс

**Ваша задача написать функцию `process_files`, которая принимает на вход два пути к папкам. Из первой папки необходимо выбрать все "чеки" (файлы по шаблону из условия), а во вторую папку сохранить один объединенный чек (отсортированный по дате, а затем по продукту) под названием `combined_data.csv`.**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить папку с файлами. После выполнения, в папке `reports_main` будут храниться все присланные магазинами чеки.

In [1]:
!wget https://github.com/vs8th/reports/archive/main.zip

import zipfile

with zipfile.ZipFile("main.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")

!rm main.zip

--2024-06-17 10:51:09--  https://github.com/vs8th/reports/archive/main.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/Vs8th/reports/zip/refs/heads/main [following]
--2024-06-17 10:51:09--  https://codeload.github.com/Vs8th/reports/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.114.10
Connecting to codeload.github.com (codeload.github.com)|140.82.114.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘main.zip’

main.zip                [ <=>                ]   3.22K  --.-KB/s    in 0s      

2024-06-17 10:51:10 (38.1 MB/s) - ‘main.zip’ saved [3296]



Чтобы посмотреть как выглядят подходящие для объединения чеки выполните следующую ячейку.

In [2]:
import pandas as pd
import pandas as pd
import datetime
import os
import re

df = pd.read_csv('reports-main/2023-02-17-05-38-2.csv', sep=";")
df

,date,product,store,cost
0,2023-02-17,product_0,store_2,10
1,2023-02-17,product_1,store_2,20
2,2023-02-17,product_2,store_2,30
3,2023-02-17,product_3,store_2,40
4,2023-02-17,product_4,store_2,50


**Решение**

Напишите свое решение ниже

**Примечание**

Не все файлы подходящие по названию, будут подходить по содержанию. Там может оказаться лишний столбец, например. Ориентируйтесь на столбцы из чека выше - это то, что вас интересует. Остальные столбцы можно просто отбросить.

**Важно**: разделителем файла на выходе должна быть запятая.

In [3]:
def process_files(src_folder, dest_folder):
  def fname_check(fname: str) -> bool:
  # Функция проверяет на соответствие имени файла заданному шаблону
    if f := re.search(r'-\d+\.csv', fname):
      try:
        datetime.datetime.strptime(fname[:f.start()],  '%Y-%m-%d-%H-%M')
        return True
      except ValueError:
        pass
    return False

  fnames = os.listdir(src_folder)
  filtered_fnames = [fname for fname in fnames if fname_check(fname)]
  if filtered_fnames:
    content = pd.read_csv(f'reports-main/{filtered_fnames[0]}', sep=";")
    columns = content.columns
    for _ in filtered_fnames[1:]:
        df = pd.read_csv(f'reports-main/{_}', sep=";")
        content = pd.concat([content, df[columns]])
    content.sort_values(by=['date', 'product'], inplace=True)
    if not os.path.exists(dest_folder):
      os.makedirs(dest_folder)
    content.to_csv(f'{dest_folder}/combined_data.csv', index=False)


src_folder = 'reports-main'
dest_folder = 'comb_reports'
process_files(src_folder, dest_folder)

✏️ ✏️ ✏️

**Проверка**

Чтобы проверить свое решение, запустите код в следующих ячейках

In [4]:
# Здесь будет скачиваться файл с эталонным ответом

!wget https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv

import pandas as pd

user_answer = pd.read_csv(f'{dest_folder}/combined_data.csv')
correct_answer = pd.read_csv('data.csv')

--2024-06-17 10:52:05--  https://gist.github.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.github.com (gist.github.com)... 140.82.112.3
Connecting to gist.github.com (gist.github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv [following]
--2024-06-17 10:52:05--  https://gist.githubusercontent.com/Vs8th/9347dd7b8f59de2997feb19770dc32c1/raw/data.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 984 [text/plain]
Saving to: ‘data.csv’

data.csv            100%[===================>]     984  --.-KB/s    in 0s      

2024-06-17 10:52:05 (30.1 MB/s) - ‘data.csv’ saved [984/984]



Ниже приведен пример проверки с использованием исходного файла data.csv, в котором в столбце date содержатся не валидные даты (2023-02-30)

In [5]:
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

AssertionError: При проверке возникла ошибка ValueError('Can only compare identically-labeled (both index and columns) DataFrame objects')

### Скачаем проверочный файл, в котором удалены строки с датой 2023-02-30 ###

In [11]:
!wget https://raw.githubusercontent.com/aleks-haksly/Simulative/main/Python/Data/data_fixed.csv

--2024-06-17 11:05:30--  https://raw.githubusercontent.com/aleks-haksly/Simulative/main/Python/Data/data_fixed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824 [text/plain]
Saving to: ‘data_fixed.csv’

data_fixed.csv      100%[===================>]     824  --.-KB/s    in 0s      

2024-06-17 11:05:30 (36.9 MB/s) - ‘data_fixed.csv’ saved [824/824]



In [12]:
correct_answer = pd.read_csv('data_fixed.csv')
try:
  assert (user_answer == correct_answer).all().all(), 'Ответы не совпадают'
  assert user_answer.columns.equals(correct_answer.columns), 'Названия столбцов не совпадают'
except Exception as err:
  raise AssertionError(f'При проверке возникла ошибка {repr(err)}')
else:
  print('Поздравляем, Вы справились и успешно прошли все проверки!')

Поздравляем, Вы справились и успешно прошли все проверки!
